# Visualizing result maps with the `geemap` Python module

author: Ghislain Vieilledent    
email: ghislain.vieilledent@cirad.fr    
web: https://ecology.ghislainv.fr    

This notebook will show you how to use the `geemap` Python module to visualize result maps obtained remotely on Google Earth Engine (GEE) or locally on your computer.

Below is a legend of the annual product from JRC

In [2]:
# Imports
import ee
import geemap

# Initialize EarthEngine
ee.Initialize()

# rgb2hex
def rgb2hex(r, g, b):
    hex = "#{:02x}{:02x}{:02x}".format(r, g, b)
    return hex

# Extent
iso3 = "REU"
if iso3 == "REU":
    extent_latlong = [55.216251, -21.389860, 55.837360, -20.871805]
elif iso3 == "MDG":
    extent_latlong = [43.10, -25.15, 51.70, -11.80]
elif iso3 == "BEN":
    extent_latlong = [0.7745, 6.2351, 3.8517, 12.4183]
elif iso3 == "NCL":
    extent_latlong = [0.7745, 6.2351, 3.8517, 12.4183]
elif iso3 == "Masoala":
    extent_latlong = [0.7745, 6.2351, 3.8517, 12.4183]

# Region
region = ee.Geometry.Rectangle(extent_latlong, proj="EPSG:4326",
                               geodesic=False)
region = region.buffer(10000).bounds()
export_coord = region.getInfo()["coordinates"]

# JRC annual product (AP)
AP = ee.ImageCollection("users/ClassifLandsat072015/Roadless2019/AnnualChanges_1982_2019")
AP = AP.mosaic().toByte().unmask()
AP = AP.clip(region)
MASK = AP.select(0)
AP = AP.where(MASK.eq(0), 0)

# ap_allYear: forest if Y = 1, 2, 3, 4, 5, 13, or 14.
AP_forest = AP.where(AP.eq(2).Or(AP.eq(3)).Or(AP.eq(4)).Or(
    AP.eq(5)).Or(AP.eq(13)).Or(AP.eq(14)), 1)
ap_allYear = AP_forest.where(AP_forest.neq(1), 0)

# Forest in Jan 2020
forest2020 = ap_allYear.select(37)

# Map.addLayer(forest2020, {'min': 0, 'max': 1, 'palette': ["red", "green"]}, "forest2020", True)

# Forest cover Jan 2015
ap_2015_2020 = ap_allYear.select(list(range(32, 38)))
forest2015 = ap_2015_2020.reduce(ee.Reducer.sum())
forest2015 = forest2015.gte(1)

# Forest cover Jan 2010
ap_2010_2020 = ap_allYear.select(list(range(27, 38)))
forest2010 = ap_2010_2020.reduce(ee.Reducer.sum())
forest2010 = forest2010.gte(1)

# Forest cover Jan 2005
ap_2005_2020 = ap_allYear.select(list(range(22, 38)))
forest2005 = ap_2005_2020.reduce(ee.Reducer.sum())
forest2005 = forest2005.gte(1)

# Forest cover Jan 2000
ap_2000_2020 = ap_allYear.select(list(range(17, 38)))
forest2000 = ap_2000_2020.reduce(ee.Reducer.sum())
forest2000 = forest2000.gte(1)

# Forest raster with five bands
forest = forest2000.addBands(forest2005).addBands(
    forest2010).addBands(forest2015).addBands(forest2020)
forest = forest.select([0, 1, 2, 3, 4], ["forest2000", "forest2005",
                                         "forest2010", "forest2015",
                                         "forest2020"])
forest = forest.set("system:bandNames", ["forest2000", "forest2005",
                                         "forest2010", "forest2015",
                                         "forest2020"])

# Forest raster with three values
forest = forest2000.where(forest2010.eq(1), 2).where(forest2020.eq(1), 3)

# Palette
Palette = [rgb2hex(255, 255, 255), rgb2hex(255, 165, 0), rgb2hex(227, 26, 28), rgb2hex(34, 139, 34)]

In [4]:
# Center and zoom
if iso3 == "REU":
    x, y, z = (55.53, -21.11, 12)
elif iso3 == "MDG":
    x, y, z = (46.70, -19.30, 8)
elif iso3 == "BEN":
    x, y, z = (0.7745, 6.2351, 8)
elif iso3 == "NCL":
    x, y, z = (0.7745, 6.2351, 8)
elif iso3 == "Masoala":
    x, y, z = (50.30, -15.47, 12)
    
# Create Map
Map = geemap.Map()
Map.setCenter(x, y, z)

# Add layer
Map.addLayer(forest.updateMask(forest), {'min': 0, 'max': 3, "palette": Palette}, "forest2000-2020", True)

# Show map
Map

Map(center=[-21.11, 55.53], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(v…